## Synthetic Dataset

In [ ]:
alpha_cfg = {
    'hotdog': (0.05, 5_000),
    'ficus': (0.05, 1_000),
    'chair': (0.1, 5_000),
    'ship': (0.05, 10_000),
    'mic': (0.1, 5_000),
    'materials': (0.06, 2_000),
    'drums': (0.05, 10_000),
    'lego': (0.05, 10_000),
}
shape = 'lego'
alpha, wt_res = alpha_cfg[shape]

In [ ]:
import torch
import trimesh
import open3d as o3d
import numpy as np
import point_cloud_utils as pcu
(model_params, _) = torch.load(f"output/synthetic_s1/{shape}/chkpnt10000.pth")
(active_sh_degree, xyz, _features_dc, _features_rest, 
_scaling, _rotation, opacity, max_radii2D, 
xyz_gradient_accum, denom,opt_dict, spatial_lr_scale) = model_params

fg_pcd = o3d.geometry.PointCloud()
fg_pcd.points = o3d.utility.Vector3dVector(xyz.detach().double().cpu().numpy())
o3d_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(fg_pcd, alpha)
vertices = np.asarray(o3d_mesh.vertices)
triangles = np.asarray(o3d_mesh.triangles)

trimesh.Trimesh(vertices, triangles).show()

In [ ]:
vw, fw = pcu.make_mesh_watertight(vertices, triangles, wt_res)
mesh = trimesh.Trimesh(vw, fw)
mesh.show()

In [ ]:
# save
path = f"output/synthetic_s1/{shape}/coarse_mesh.obj"
with open(path, 'w') as file:
    mesh.export(file_obj=file, file_type='obj')

## Self-Dyn dataset

In [ ]:
alpha_cfg = {
    'lego_init': (0.05, 10_000),
}
shape = 'lego_init'
alpha, wt_res = alpha_cfg[shape]


import torch
import trimesh
import open3d as o3d
import numpy as np
import point_cloud_utils as pcu

(model_params, _) = torch.load(f"output_dyn/ours_s1/{shape}/chkpnt10000.pth")
(active_sh_degree, xyz, _features_dc, _features_rest, 
_scaling, _rotation, opacity, max_radii2D, 
xyz_gradient_accum, denom,opt_dict, spatial_lr_scale) = model_params

fg_pcd = o3d.geometry.PointCloud()
fg_pcd.points = o3d.utility.Vector3dVector(xyz.detach().double().cpu().numpy())
o3d_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(fg_pcd, alpha)
vertices = np.asarray(o3d_mesh.vertices)
triangles = np.asarray(o3d_mesh.triangles)

trimesh.Trimesh(vertices, triangles).show()

In [ ]:
vw, fw = pcu.make_mesh_watertight(vertices, triangles, wt_res)
mesh = trimesh.Trimesh(vw, fw)
mesh.show()

In [ ]:
# save
path = f"output_dyn/ours_s1/{shape}/coarse_mesh.obj"
with open(path, 'w') as file:
    mesh.export(file_obj=file, file_type='obj')

## Mipnerf360 Dataset

In [ ]:
alpha_cfg = {
    'room': (0.1, 50_000, 1.8),
    'kitchen': (0.1, 80_000, 5.0),
    'bicycle': (0.1, 80_000, 4.0),
    'garden': (0.1, 80_000, 4.0),
    'stump': (0.1, 50_000, 4.0),
    'counter': (0.1, 50_000, 4.0),
    'bonsai': (0.1, 50_000, 4.0),
}
min_max_cfg = {
    'room': [[-4.3, -4.5, -11.8], [4.7, 8.5, 7.0],
             [-4.3, -4.5, -11.8], [4.7, 8.5, 7.0]],  # fg, bg the same.
    'kitchen': [[[-4.2, -3., -4.3], [5., 5., 5.]],
                [[-18.5, -12, -18], [18.5, 15, 17]]],
    # 'bicycle': [[[-4, -0.3, -4], [3.5, 3.3, 4]], [[-19., -15, -19.], [19., 6, 19.]]],
    'bicycle': [[[-0.9, -0.3, -1.5], [2.0, 2.5, 2.5]],
                [[-19., -15, -19.], [19., 6, 19.]]],
    'garden': [[[-4.5, -0.1, -3.5], [3.5, 4, 5]],
               [[-13., -9, -15.], [20, 9, 15.]]],
    'stump': [[[-4, -1., -1.], [4, 4, 4]],
              [[-19.5, -19.5, -19.5], [19.5, 19.5, 19.5]]],
    'counter': [[[-3, -1., -5], [5., 5, 5]],
                [[-12, -8, -10], [13, 13, 17]]],
    'bonsai': [[[-2, 0, 0], [2, 5., 5.]],
               [[-20, -13, -13], [15, 17, 20]]]
}
shape = 'bicycle'
alpha, wt_res, scale_rate = alpha_cfg[shape]
fg_bg_min_max_raw = min_max_cfg[shape]

In [ ]:
import torch
import trimesh
import open3d as o3d
import numpy as np
import point_cloud_utils as pcu
(model_params, _) = torch.load(f"output/mipnerf360_s1/{shape}/chkpnt10000.pth")
(active_sh_degree, xyz, _features_dc, _features_rest, 
_scaling, _rotation, opacity, max_radii2D, 
xyz_gradient_accum, denom,opt_dict, spatial_lr_scale) = model_params
device = xyz.device
print(spatial_lr_scale)

if shape == 'stump':  # TODO: apply to all shapes?
    xyz = xyz[torch.sigmoid(opacity[:, 0])>0.3]

# ref
scene_scale_radius = scale_rate * spatial_lr_scale
scene_scale = scene_scale_radius*2
xyz_min = torch.tensor([[-0.5, -0.5, -0.5]], device=device) * scene_scale
xyz_max = torch.tensor([[0.5, 0.5, 0.5]], device=device) * scene_scale

# our own
fg_bg_min_max = torch.tensor(fg_bg_min_max_raw, device=device).view(2, 2, 3)
xyz_min, xyz_max = fg_bg_min_max[1]  # bg

# filter xyz
inbox_mask = (xyz_min<xyz).all(dim=1) & (xyz<xyz_max).all(dim=1)
xyz = xyz[inbox_mask]
print(xyz_max)

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz.detach().double().cpu().numpy())
# cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=20.)
# pcd = pcd.select_by_index(ind)
o3d_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
vertices = np.asarray(o3d_mesh.vertices)
triangles = np.asarray(o3d_mesh.triangles)

# trimesh.Trimesh(vertices, triangles).show()

In [ ]:
print('start')
vw, fw = pcu.make_mesh_watertight(vertices, triangles, wt_res)
mesh = trimesh.Trimesh(vw, fw)
# mesh.show()

In [ ]:
scene = trimesh.Scene()
scene.add_geometry(mesh)

def get_box(s_min, s_max):
    draw_verts = np.array([
        [s_min[0], s_min[1], s_min[2]], [s_max[0], s_min[1], s_min[2]], 
        [s_min[0], s_max[1], s_min[2]], [s_max[0], s_max[1], s_min[2]], 
        [s_min[0], s_min[1], s_max[2]], [s_max[0], s_min[1], s_max[2]], 
        [s_min[0], s_max[1], s_max[2]], [s_max[0], s_max[1], s_max[2]], 
    ])
    el = trimesh.path.entities.Line([0,1,0,2,3,1,5,7,3,2,6,7,5,4,6,4,0])
    path3d = trimesh.path.Path3D(entities=[el], vertices=draw_verts)
    path3d.colors = [[255, 0, 0, 255]]  # RGBA
    return path3d
fg_bg_min_max = fg_bg_min_max.to('cpu')
scene.add_geometry(get_box(fg_bg_min_max[0,0], fg_bg_min_max[0,1]))  # fg
scene.add_geometry(get_box(fg_bg_min_max[1,0], fg_bg_min_max[1,1]))  # bg
axis = trimesh.creation.axis(origin_size=1.0, origin_color=[0, 0, 255, 255])  # Blue origin
scene.add_geometry(axis)

scene.show()

In [ ]:
# save
path = f"output/mipnerf360_s1/{shape}/coarse_mesh.obj"
with open(path, 'w') as file:
    mesh.export(file_obj=file, file_type='obj')

In [ ]:
import trimesh
trimesh.load_mesh(f"output/mipnerf360_s1/kitchen/coarse_mesh.obj")